# Prepare DES transient list with relevant metadata

This notebook will take DES SN data and prepare a list for applying various cuts (`Apply_Cuts_vX.ipynb`) to produce an SNID name list, which can be passed to PSNID for fitting and assessment.  The final list can be parsed to `MAKE_TEMPLATES/GP2-code-vX.ipynb` to produce SED time-series templates for SNANA.

In [1]:
import os.path
import numpy as np
import pandas as pd
from astropy.io import ascii

In [2]:
BASEDIR = '/Users/masao/dessn/'

# Workflow
**(This code)**
* Take output from `01_LightCurveStats.ipynb` and `SNANA`
  * `BASEDIR + 'LC_MergedOutput.csv'`
  * `BASEDIR + 'sntable_append_DES.csv'`

* Compute several new flags
  * `N_ml_flag` = number of seasons with more that 5 epochs that pass ML cuts
  * `N_ml_flag2` : 1 = single season, 2 = two adjacent seasons, 0 = otherwise
  * `Year` = year with largest N_ml_YX value
  * `yearVARX` = year with largest variability
  * `VARX_flag` = 1 = single season, 2 = two adjacent seasons, 0 = otherwise
* Merge with `TFLAG` (transient status)

**(Then 03_ApplyCuts_vX.ipynb)**

## SNANA

We first must obtain the output data from SNANA. A directory has been created for you on midway2
/project2/rkessler/SURVEYS/DES/USERS/rhounsell/STUDENT/HOW_TO_CUT/

If you  run
    
    snana.exe  des_5yr_base.nml
    
    This creates DES.HBOOK and DES.SNANA.TEXT
    You then need to run the following commands to get the number of ML epochs out.
    
    sntable_dump.pl DES.HBOOK SNANA --v DES_numepochs_ml DES_numepochs_ml_Y0 DES_numepochs_ml_Y1 
    DES_numepochs_ml_Y2 DES_numepochs_ml_Y3 DES_numepochs_ml_Y4 DES_numepochs_ml_Y5  --a
    DES.SNANA.TEXT

    
    This will produce  sntable_append_DES.text - you need this for the follwing code.
    
    
Note to run this code you will have to replace /Users/hounsell/Desktop/DES/STUDENTS/ with your own directory

In [3]:
# Read in data files

# Output from 01_LightCurveStats.ipynb
data0 = pd.read_csv(BASEDIR + 'LC_MergedOutput.csv')
# SNANA info
data1 = pd.read_csv(BASEDIR + 'sntable_append_DES.csv')

In [4]:
data0

,CID,TYPE,FIELD,zHD,zHDERR,MWEBV,HOST_NMATCH,HOST_NMATCH2,HOST_OBJID,HOST_ZPHOT,...,peakmjd_i,peakmjd_z,nnondet_g,nnondet_r,nnondet_i,nnondet_z,firstdetday_g,firstdetday_r,firstdetday_i,firstdetday_z
0,1246273,0,C1,2.3522,0.0010,0.010097,1,1,590,1.43312,...,56543.314,56534.225,0,0,0,0,29.218,29.221,29.223,29.225
1,1246344,0,C1,-9.0000,-9.0000,0.012869,1,2,337,1.02155,...,56590.345,56590.348,0,0,0,0,29.218,29.221,29.223,29.225
2,1246433,0,X3,1.2206,0.0007,0.027303,1,1,19800,0.28819,...,57365.081,57317.097,0,0,0,0,14.293,14.302,14.318,14.341
3,1246515,0,X3,-9.0000,-9.0000,0.025877,1,1,18292,0.78169,...,56537.379,56563.339,0,0,0,0,31.207,33.261,32.379,32.291
4,1246618,0,X3,-9.0000,-9.0000,0.026634,1,3,156526,0.46001,...,56537.379,56537.291,20,18,0,0,0.000,0.000,32.379,32.291
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31631,1965102,0,C1,-9.0000,-9.0000,0.010881,1,1,153121,0.82337,...,58004.375,58139.050,31,31,31,31,0.000,0.000,0.000,0.000
31632,1968025,0,C2,-9.0000,-9.0000,0.014389,2,4,-58337,-9.00000,...,58026.325,58041.136,30,29,30,29,0.000,0.000,0.000,0.000
31633,1970027,0,C2,-9.0000,-9.0000,0.012923,1,1,153398,0.43243,...,58026.325,58026.330,30,28,30,16,0.000,0.000,0.000,0.000
31634,1979080,0,C3,-9.0000,-9.0000,0.008402,1,1,153596,0.71025,...,57998.249,58026.307,27,28,27,25,0.000,0.000,0.000,0.000


In [5]:
data0.columns

Index(['CID', 'TYPE', 'FIELD', 'zHD', 'zHDERR', 'MWEBV', 'HOST_NMATCH',
       'HOST_NMATCH2', 'HOST_OBJID', 'HOST_ZPHOT', 'HOST_ZPHOTERR',
       'HOST_ZSPEC', 'HOST_ZSPECERR', 'HOST_RA', 'HOST_DEC', 'HOST_ANGSEP',
       'HOST_DDLR', 'HOST_CONFUSION', 'HOST_LOGMASS', 'HOST_LOGMASS_ERR',
       'SNRMAX1', 'SNRMAX2', 'SNRMAX3', 'TRANSIENT_NAME', 'RA', 'DEC',
       'nSNR3_Y0', 'nSNR3_Y1', 'nSNR3_Y2', 'nSNR3_Y3', 'nSNR3_Y4', 'nSNR3_Y5',
       'yearSNR3', 'flagSNR3', 'nSNR5_Y0', 'nSNR5_Y1', 'nSNR5_Y2', 'nSNR5_Y3',
       'nSNR5_Y4', 'nSNR5_Y5', 'yearSNR5', 'flagSNR5', 'cumSNR_Y0',
       'cumSNR_Y1', 'cumSNR_Y2', 'cumSNR_Y3', 'cumSNR_Y4', 'cumSNR_Y5',
       'cumSNRsgn_Y0', 'cumSNRsgn_Y1', 'cumSNRsgn_Y2', 'cumSNRsgn_Y3',
       'cumSNRsgn_Y4', 'cumSNRsgn_Y5', 'VAR0_Y0', 'VAR0_Y1', 'VAR0_Y2',
       'VAR0_Y3', 'VAR0_Y4', 'VAR0_Y5', 'VAR1_Y0', 'VAR1_Y1', 'VAR1_Y2',
       'VAR1_Y3', 'VAR1_Y4', 'VAR1_Y5', 'VARX_Y0', 'VARX_Y1', 'VARX_Y2',
       'VARX_Y3', 'VARX_Y4', 'VARX_Y5', 'minmag_g', 

In [6]:
data1

,CID,CIDint,IDSURVEY,TYPE,FIELD,CUTFLAG_SNANA,zHEL,zHELERR,zCMB,zCMBERR,...,SNRMAX1,SNRMAX2,SNRMAX3,DES_numepochs_ml,DES_numepochs_ml_Y0,DES_numepochs_ml_Y1,DES_numepochs_ml_Y2,DES_numepochs_ml_Y3,DES_numepochs_ml_Y4,DES_numepochs_ml_Y5
0,1246273,1246273,10,0,C1,1,2.3534,0.001,2.3522,0.001,...,13.33338,11.76433,-9.00000,28,0,28,0,12,32,16
1,1246274,1246274,10,0,C1,1,0.1951,0.001,0.1947,0.001,...,25.85307,19.65114,15.17255,16,0,12,0,6,14,31
2,1246275,1246275,10,0,C1,1,0.2465,0.001,0.2460,0.001,...,38.93864,30.07702,19.15641,13,0,13,0,0,0,0
3,1246276,1246276,10,0,C1,1,-9.0000,-9.000,-9.0000,-9.000,...,12.42150,6.55736,5.45821,8,0,5,2,1,0,0
4,1246278,1246278,10,0,C1,1,-9.0000,-9.000,-9.0000,-9.000,...,22.92678,16.83920,13.33784,12,0,12,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31631,1997029,1997029,10,0,C2,0,-9.0000,-9.000,-9.0000,-9.000,...,3.64850,3.57797,3.54046,2,0,1,0,1,0,0
31632,1997033,1997033,10,0,C2,0,-9.0000,-9.000,-9.0000,-9.000,...,2.77177,2.38396,2.01580,2,0,0,0,0,1,2
31633,1998019,1998019,10,0,C2,1,-9.0000,-9.000,-9.0000,-9.000,...,5.20589,4.01121,3.28144,2,0,0,0,0,0,4
31634,1999022,1999022,10,0,C2,0,0.1046,0.001,0.1042,0.001,...,3.78108,2.07794,2.02490,2,0,1,0,0,0,1


In [7]:
data1.columns

Index(['CID', 'CIDint', 'IDSURVEY', 'TYPE', 'FIELD', 'CUTFLAG_SNANA', 'zHEL',
       'zHELERR', 'zCMB', 'zCMBERR', 'zHD', 'zHDERR', 'VPEC', 'VPECERR',
       'MWEBV', 'HOST_LOGMASS', 'HOST_LOGMASS_ERR', 'HOST_sSFR',
       'HOST_sSFR_ERR', 'PKMJDINI', 'SNRMAX1', 'SNRMAX2', 'SNRMAX3',
       'DES_numepochs_ml', 'DES_numepochs_ml_Y0', 'DES_numepochs_ml_Y1',
       'DES_numepochs_ml_Y2', 'DES_numepochs_ml_Y3', 'DES_numepochs_ml_Y4',
       'DES_numepochs_ml_Y5'],
      dtype='object')

In [8]:
df1 = data0.merge(data1, on='CID', how='outer', suffixes=['','_remove'])
df1 = df1.drop(['TYPE_remove', 'FIELD_remove',
                'zHD_remove', 'zHDERR_remove','MWEBV_remove',
                'HOST_LOGMASS_remove', 'HOST_LOGMASS_ERR_remove',
                'HOST_sSFR', 'HOST_sSFR_ERR', 'PKMJDINI', 'SNRMAX1_remove',
                'SNRMAX2_remove', 'SNRMAX3_remove'], axis=1)

In [9]:
df1 = df1.rename(columns={'DES_numepochs_ml': 'N_ml',
                          'DES_numepochs_ml_Y0': 'N_ml_Y0',
                          'DES_numepochs_ml_Y1': 'N_ml_Y1',
                          'DES_numepochs_ml_Y2': 'N_ml_Y2',
                          'DES_numepochs_ml_Y3': 'N_ml_Y3',
                          'DES_numepochs_ml_Y4': 'N_ml_Y4',
                          'DES_numepochs_ml_Y5': 'N_ml_Y5'})

In [10]:
df1.columns

Index(['CID', 'TYPE', 'FIELD', 'zHD', 'zHDERR', 'MWEBV', 'HOST_NMATCH',
       'HOST_NMATCH2', 'HOST_OBJID', 'HOST_ZPHOT',
       ...
       'zCMBERR', 'VPEC', 'VPECERR', 'N_ml', 'N_ml_Y0', 'N_ml_Y1', 'N_ml_Y2',
       'N_ml_Y3', 'N_ml_Y4', 'N_ml_Y5'],
      dtype='object', length=108)

## `N_ml` flags

In [11]:
yflags = []
for a in range(len(df1)):
    yflag  = -1
    maxnml = -1
    if df1['N_ml_Y0'][a] > maxnml:
        yflag  = 0
        maxnml = df1['N_ml_Y0'][a]
    if df1['N_ml_Y1'][a] > maxnml:
        yflag  = 1
        maxnml = df1['N_ml_Y1'][a]
    if df1['N_ml_Y2'][a] > maxnml:
        yflag  = 2
        maxnml = df1['N_ml_Y2'][a]
    if df1['N_ml_Y3'][a] > maxnml:
        yflag  = 3
        maxnml = df1['N_ml_Y3'][a]
    if df1['N_ml_Y4'][a] > maxnml:
        yflag  = 4
        maxnml = df1['N_ml_Y4'][a]
    if df1['N_ml_Y5'][a] > maxnml:
        yflag  = 5
        maxnml = df1['N_ml_Y5'][a]
    yflags.append(yflag)

In [12]:
len(yflags)

31636

In [13]:
df1['Year'] = yflags

In [14]:
df1

,CID,TYPE,FIELD,zHD,zHDERR,MWEBV,HOST_NMATCH,HOST_NMATCH2,HOST_OBJID,HOST_ZPHOT,...,VPEC,VPECERR,N_ml,N_ml_Y0,N_ml_Y1,N_ml_Y2,N_ml_Y3,N_ml_Y4,N_ml_Y5,Year
0,1246273,0,C1,2.3522,0.0010,0.010097,1,1,590,1.43312,...,0,0,28,0,28,0,12,32,16,4
1,1246344,0,C1,-9.0000,-9.0000,0.012869,1,2,337,1.02155,...,0,0,32,0,28,0,18,22,8,1
2,1246433,0,X3,1.2206,0.0007,0.027303,1,1,19800,0.28819,...,0,0,17,0,6,0,49,0,0,3
3,1246515,0,X3,-9.0000,-9.0000,0.025877,1,1,18292,0.78169,...,0,0,23,0,23,0,0,0,0,1
4,1246618,0,X3,-9.0000,-9.0000,0.026634,1,3,156526,0.46001,...,0,0,2,0,2,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31631,1965102,0,C1,-9.0000,-9.0000,0.010881,1,1,153121,0.82337,...,0,0,2,0,0,0,0,0,1,5
31632,1968025,0,C2,-9.0000,-9.0000,0.014389,2,4,-58337,-9.00000,...,0,0,2,0,0,0,0,0,2,5
31633,1970027,0,C2,-9.0000,-9.0000,0.012923,1,1,153398,0.43243,...,0,0,2,0,0,0,0,0,2,5
31634,1979080,0,C3,-9.0000,-9.0000,0.008402,1,1,153596,0.71025,...,0,0,2,0,0,0,0,0,2,5


In [15]:
# Flag transients with >=5 ML epochs

flag = np.zeros(6,dtype=int)
sum_ff = []

for a in range(len(df1)):
    flag[0] = df1['N_ml_Y0'][a]>=4.99
    flag[1] = df1['N_ml_Y1'][a]>=4.99
    flag[2] = df1['N_ml_Y2'][a]>=4.99
    flag[3] = df1['N_ml_Y3'][a]>=4.99
    flag[4] = df1['N_ml_Y4'][a]>=4.99
    flag[5] = df1['N_ml_Y5'][a]>=4.99
    sum_f = np.sum(flag)
#    print(flag,sum_f)
    sum_ff.append(sum_f)

In [16]:
# Now we need to know where sum_ff  = 1, as we dont want detections in multiple seasons.
sum_ff = np.array(sum_ff)
df1['N_ml_flag'] = sum_ff
# N_ml_flag = number of seasons in which N_ml_YX >= 5

In [17]:
# Flag transients with two adjacent seaons with 3 ML epochs or more.

flag = np.zeros(6,dtype=int)
sum_ff = []

for a in range(len(df1)):
    flag[0] = df1['N_ml_Y0'][a]>=2.99
    flag[1] = df1['N_ml_Y1'][a]>=2.99
    flag[2] = df1['N_ml_Y2'][a]>=2.99
    flag[3] = df1['N_ml_Y3'][a]>=2.99
    flag[4] = df1['N_ml_Y4'][a]>=2.99
    flag[5] = df1['N_ml_Y5'][a]>=2.99
    flag[1] = 2*flag[1]
    flag[2] = 4*flag[2]
    flag[3] = 8*flag[3]
    flag[4] = 16*flag[4]
    flag[5] = 32*flag[5]
    sum_f = np.sum(flag)
#    print(flag, sum_f,
#          df1['N_ml_Y0'][a], df1['N_ml_Y1'][a], df1['N_ml_Y2'][a],
#          df1['N_ml_Y3'][a], df1['N_ml_Y4'][a], df1['N_ml_Y5'][a])
    sum_ff.append(sum_f)

In [18]:
thisflags = []

for a in range(len(sum_ff)):
    if (sum_ff[a] == 1) | (sum_ff[a] == 2) | (sum_ff[a] == 4) | (sum_ff[a] == 8) | (sum_ff[a] == 16) | (sum_ff[a] == 32):
        # single season transient
        thisflag = 1
    elif (sum_ff[a] == 3) | (sum_ff[a] == 6) | (sum_ff[a] == 12) | (sum_ff[a] == 24) | (sum_ff[a] == 48):
        # 2-season transient in adjacent years
        thisflag = 2
    else:
        thisflag = 0

    thisflags.append(thisflag)

In [19]:
# append to df1
df1['N_ml_flag2'] = thisflags

In [20]:
df1

,CID,TYPE,FIELD,zHD,zHDERR,MWEBV,HOST_NMATCH,HOST_NMATCH2,HOST_OBJID,HOST_ZPHOT,...,N_ml,N_ml_Y0,N_ml_Y1,N_ml_Y2,N_ml_Y3,N_ml_Y4,N_ml_Y5,Year,N_ml_flag,N_ml_flag2
0,1246273,0,C1,2.3522,0.0010,0.010097,1,1,590,1.43312,...,28,0,28,0,12,32,16,4,4,0
1,1246344,0,C1,-9.0000,-9.0000,0.012869,1,2,337,1.02155,...,32,0,28,0,18,22,8,1,4,0
2,1246433,0,X3,1.2206,0.0007,0.027303,1,1,19800,0.28819,...,17,0,6,0,49,0,0,3,2,0
3,1246515,0,X3,-9.0000,-9.0000,0.025877,1,1,18292,0.78169,...,23,0,23,0,0,0,0,1,1,1
4,1246618,0,X3,-9.0000,-9.0000,0.026634,1,3,156526,0.46001,...,2,0,2,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31631,1965102,0,C1,-9.0000,-9.0000,0.010881,1,1,153121,0.82337,...,2,0,0,0,0,0,1,5,0,0
31632,1968025,0,C2,-9.0000,-9.0000,0.014389,2,4,-58337,-9.00000,...,2,0,0,0,0,0,2,5,0,0
31633,1970027,0,C2,-9.0000,-9.0000,0.012923,1,1,153398,0.43243,...,2,0,0,0,0,0,2,5,0,0
31634,1979080,0,C3,-9.0000,-9.0000,0.008402,1,1,153596,0.71025,...,2,0,0,0,0,0,2,5,0,0


## VAR flags

In [21]:
yflags = []
for a in range(len(df1)):
    yflag  = -1
    maxvar = -1
    if df1['VARX_Y0'][a] > maxvar:
        yflag  = 0
        maxvar = df1['VARX_Y0'][a]
    if df1['VARX_Y1'][a] > maxvar:
        yflag  = 1
        maxvar = df1['VARX_Y1'][a]
    if df1['VARX_Y2'][a] > maxvar:
        yflag  = 2
        maxvar = df1['VARX_Y2'][a]
    if df1['VARX_Y3'][a] > maxvar:
        yflag  = 3
        maxvar = df1['VARX_Y3'][a]
    if df1['VARX_Y4'][a] > maxvar:
        yflag  = 4
        maxvar = df1['VARX_Y4'][a]
    if df1['VARX_Y5'][a] > maxvar:
        yflag  = 5
        maxvar = df1['VARX_Y5'][a]
    yflags.append(yflag)

In [22]:
len(yflags)

31636

In [23]:
df1['yearVARX'] = yflags

In [24]:
df1

,CID,TYPE,FIELD,zHD,zHDERR,MWEBV,HOST_NMATCH,HOST_NMATCH2,HOST_OBJID,HOST_ZPHOT,...,N_ml_Y0,N_ml_Y1,N_ml_Y2,N_ml_Y3,N_ml_Y4,N_ml_Y5,Year,N_ml_flag,N_ml_flag2,yearVARX
0,1246273,0,C1,2.3522,0.0010,0.010097,1,1,590,1.43312,...,0,28,0,12,32,16,4,4,0,1
1,1246344,0,C1,-9.0000,-9.0000,0.012869,1,2,337,1.02155,...,0,28,0,18,22,8,1,4,0,1
2,1246433,0,X3,1.2206,0.0007,0.027303,1,1,19800,0.28819,...,0,6,0,49,0,0,3,2,0,5
3,1246515,0,X3,-9.0000,-9.0000,0.025877,1,1,18292,0.78169,...,0,23,0,0,0,0,1,1,1,1
4,1246618,0,X3,-9.0000,-9.0000,0.026634,1,3,156526,0.46001,...,0,2,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31631,1965102,0,C1,-9.0000,-9.0000,0.010881,1,1,153121,0.82337,...,0,0,0,0,0,1,5,0,0,5
31632,1968025,0,C2,-9.0000,-9.0000,0.014389,2,4,-58337,-9.00000,...,0,0,0,0,0,2,5,0,0,2
31633,1970027,0,C2,-9.0000,-9.0000,0.012923,1,1,153398,0.43243,...,0,0,0,0,0,2,5,0,0,5
31634,1979080,0,C3,-9.0000,-9.0000,0.008402,1,1,153596,0.71025,...,0,0,0,0,0,2,5,0,0,4


In [25]:
# Flag transients with >=5-sigma coefficients in light curve

flag = np.zeros(6,dtype=int)
sum_ff = []

for a in range(len(df1)):
    flag[0] = df1['VARX_Y0'][a]>=4.99
    flag[1] = df1['VARX_Y1'][a]>=4.99
    flag[2] = df1['VARX_Y2'][a]>=4.99
    flag[3] = df1['VARX_Y3'][a]>=4.99
    flag[4] = df1['VARX_Y4'][a]>=4.99
    flag[5] = df1['VARX_Y5'][a]>=4.99
    sum_f = np.sum(flag)
#    print(flag,sum_f)
    sum_ff.append(sum_f)

In [26]:
# VAR1_flag = number of seasons in which N_ml_YX >= 5
sum_ff = np.array(sum_ff)
df1['VARX_flag'] = sum_ff

In [27]:
df1

,CID,TYPE,FIELD,zHD,zHDERR,MWEBV,HOST_NMATCH,HOST_NMATCH2,HOST_OBJID,HOST_ZPHOT,...,N_ml_Y1,N_ml_Y2,N_ml_Y3,N_ml_Y4,N_ml_Y5,Year,N_ml_flag,N_ml_flag2,yearVARX,VARX_flag
0,1246273,0,C1,2.3522,0.0010,0.010097,1,1,590,1.43312,...,28,0,12,32,16,4,4,0,1,5
1,1246344,0,C1,-9.0000,-9.0000,0.012869,1,2,337,1.02155,...,28,0,18,22,8,1,4,0,1,5
2,1246433,0,X3,1.2206,0.0007,0.027303,1,1,19800,0.28819,...,6,0,49,0,0,3,2,0,5,5
3,1246515,0,X3,-9.0000,-9.0000,0.025877,1,1,18292,0.78169,...,23,0,0,0,0,1,1,1,1,3
4,1246618,0,X3,-9.0000,-9.0000,0.026634,1,3,156526,0.46001,...,2,0,0,0,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31631,1965102,0,C1,-9.0000,-9.0000,0.010881,1,1,153121,0.82337,...,0,0,0,0,1,5,0,0,5,0
31632,1968025,0,C2,-9.0000,-9.0000,0.014389,2,4,-58337,-9.00000,...,0,0,0,0,2,5,0,0,2,0
31633,1970027,0,C2,-9.0000,-9.0000,0.012923,1,1,153398,0.43243,...,0,0,0,0,2,5,0,0,5,4
31634,1979080,0,C3,-9.0000,-9.0000,0.008402,1,1,153596,0.71025,...,0,0,0,0,2,5,0,0,4,1


In [28]:
# Flag transients with two adjacent seaons with 5-sigma polynomial coeff

flag = np.zeros(6,dtype=int)
sum_ff = []

for a in range(len(df1)):
    flag[0] = df1['VARX_Y0'][a]>=5.00
    flag[1] = df1['VARX_Y1'][a]>=5.00
    flag[2] = df1['VARX_Y2'][a]>=5.00
    flag[3] = df1['VARX_Y3'][a]>=5.00
    flag[4] = df1['VARX_Y4'][a]>=5.00
    flag[5] = df1['VARX_Y5'][a]>=5.00
    flag[1] = 2*flag[1]
    flag[2] = 4*flag[2]
    flag[3] = 8*flag[3]
    flag[4] = 16*flag[4]
    flag[5] = 32*flag[5]
    sum_f = np.sum(flag)
#    print(flag, sum_f,
#          df1['N_ml_Y0'][a], df1['N_ml_Y1'][a], df1['N_ml_Y2'][a],
#          df1['N_ml_Y3'][a], df1['N_ml_Y4'][a], df1['N_ml_Y5'][a])
    sum_ff.append(sum_f)

In [29]:
thisflags = []

for a in range(len(sum_ff)):
    if (sum_ff[a] == 1) | (sum_ff[a] == 2) | (sum_ff[a] == 4) | (sum_ff[a] == 8) | (sum_ff[a] == 16) | (sum_ff[a] == 32):
        # single season transient
        thisflag = 1
    elif (sum_ff[a] == 3) | (sum_ff[a] == 6) | (sum_ff[a] == 12) | (sum_ff[a] == 24) | (sum_ff[a] == 48):
        # 2-season transient in adjacent years
        thisflag = 2
    else:
        thisflag = 0

    thisflags.append(thisflag)

In [30]:
# append to df1
df1['VARX_flag2'] = thisflags

In [31]:
df1[['CID',
     'VARX_Y0', 'VARX_Y1', 'VARX_Y2', 'VARX_Y3', 'VARX_Y4', 'VARX_Y5', 'VARX_flag', 'VARX_flag2',
     'flagSNR3', 'flagSNR5']][df1['TYPE']==1]

,CID,VARX_Y0,VARX_Y1,VARX_Y2,VARX_Y3,VARX_Y4,VARX_Y5,VARX_flag,VARX_flag2,flagSNR3,flagSNR5
97,1257366,0.0,3.763054,39.558573,4.967465,2.246880,4.310997,1,1,1,1
236,1292315,0.0,51.838805,17.367217,44.931999,43.598150,37.939877,5,0,1,1
249,1294014,0.0,28.551254,15.852375,27.629217,16.160809,17.515138,5,0,1,1
426,1282757,0.0,3.322810,2.842840,15.897000,3.445742,0.848783,1,1,1,1
432,1283936,0.0,3.000267,2.224901,17.972092,5.894149,3.027630,2,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...
31088,1336008,0.0,5.674245,2.186525,6.650813,5.495302,2.871887,3,0,1,1
31100,1338278,0.0,4.134971,2.643418,7.874934,3.131163,3.873763,1,1,1,1
31127,1343337,0.0,3.452080,3.879148,10.733394,4.587088,4.383356,1,1,1,1
31163,1368131,0.0,6.186469,6.178814,5.743104,22.285838,7.866185,5,0,1,1


In [32]:
testdf = df1[(df1['VARX_flag']!=1) & (df1['TYPE']==1)]
len(testdf)

211

## Tests

In [33]:
df1[['CID',
     'nSNR5_Y0', 'nSNR5_Y1', 'nSNR5_Y2', 'nSNR5_Y3', 'nSNR5_Y4', 'nSNR5_Y5', 'yearSNR5', 'flagSNR5',
     'N_ml_Y0', 'N_ml_Y1', 'N_ml_Y2', 'N_ml_Y3', 'N_ml_Y4', 'N_ml_Y5', 'Year', 'N_ml_flag', 'N_ml_flag2', 
    ]][df1['CID']==1433017]

,CID,nSNR5_Y0,nSNR5_Y1,nSNR5_Y2,nSNR5_Y3,nSNR5_Y4,nSNR5_Y5,yearSNR5,flagSNR5,N_ml_Y0,N_ml_Y1,N_ml_Y2,N_ml_Y3,N_ml_Y4,N_ml_Y5,Year,N_ml_flag,N_ml_flag2
18580,1433017,0,0,0,0,1,52,5,1,0,0,0,0,15,18,5,2,2


In [34]:
df1[['CID',
     'nSNR5_Y0', 'nSNR5_Y1', 'nSNR5_Y2', 'nSNR5_Y3', 'nSNR5_Y4', 'nSNR5_Y5', 'yearSNR5', 'flagSNR5',
     'N_ml_Y0', 'N_ml_Y1', 'N_ml_Y2', 'N_ml_Y3', 'N_ml_Y4', 'N_ml_Y5', 'Year', 'N_ml_flag', 'N_ml_flag2', 
    ]][df1['CID']==1879525]

,CID,nSNR5_Y0,nSNR5_Y1,nSNR5_Y2,nSNR5_Y3,nSNR5_Y4,nSNR5_Y5,yearSNR5,flagSNR5,N_ml_Y0,N_ml_Y1,N_ml_Y2,N_ml_Y3,N_ml_Y4,N_ml_Y5,Year,N_ml_flag,N_ml_flag2
20911,1879525,0,0,0,0,0,113,5,1,0,0,0,0,0,19,5,1,1


## Finally, add Transient Status flag

In [35]:
ts = ascii.read(BASEDIR + 'TransientStatus_t1.txt')
ts1 = pd.DataFrame({'CID' : np.array(ts['CID']),
                    'TFLAG' : np.array(ts['FLAG'])
                   })

In [36]:
df2 = pd.merge(df1, ts1, on='CID', how='left')
df2 = df2.astype({'TFLAG': 'Int64'})
df2

,CID,TYPE,FIELD,zHD,zHDERR,MWEBV,HOST_NMATCH,HOST_NMATCH2,HOST_OBJID,HOST_ZPHOT,...,N_ml_Y3,N_ml_Y4,N_ml_Y5,Year,N_ml_flag,N_ml_flag2,yearVARX,VARX_flag,VARX_flag2,TFLAG
0,1246273,0,C1,2.3522,0.0010,0.010097,1,1,590,1.43312,...,12,32,16,4,4,0,1,5,0,1
1,1246344,0,C1,-9.0000,-9.0000,0.012869,1,2,337,1.02155,...,18,22,8,1,4,0,1,5,0,1
2,1246433,0,X3,1.2206,0.0007,0.027303,1,1,19800,0.28819,...,49,0,0,3,2,0,5,5,0,5
3,1246515,0,X3,-9.0000,-9.0000,0.025877,1,1,18292,0.78169,...,0,0,0,1,1,1,1,3,0,1
4,1246618,0,X3,-9.0000,-9.0000,0.026634,1,3,156526,0.46001,...,0,0,0,1,0,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31631,1965102,0,C1,-9.0000,-9.0000,0.010881,1,1,153121,0.82337,...,0,0,1,5,0,0,5,0,0,16
31632,1968025,0,C2,-9.0000,-9.0000,0.014389,2,4,-58337,-9.00000,...,0,0,2,5,0,0,2,0,0,16
31633,1970027,0,C2,-9.0000,-9.0000,0.012923,1,1,153398,0.43243,...,0,0,2,5,0,0,5,4,0,16
31634,1979080,0,C3,-9.0000,-9.0000,0.008402,1,1,153596,0.71025,...,0,0,2,5,0,0,4,1,1,0


### Summary of added flags
* `N_ml_flag` = number of seasons with more than 3 epochs that pass ML cuts [0,5]
* `N_ml_flag2` : 1 = single season, 2 = two adjacent seasons, 0 = otherwise [1,2,0]
* `Year` = year with largest N_ml [0,5]
* `TFLAG` = transient status [0,16]
* `VARl_flag` = number of seasons with >5-sigma slope in light curve [0,5]
* `VAR1_flag2` = 1 = single season, 2 = two adjacent seasons, 0 = otherwise [1,2,0]

In [37]:
fname = BASEDIR + 'DEStransientsMaster.csv'

if os.path.exists(fname):
    os.remove(fname)
    
df2.to_csv(fname, index=False)

In [38]:
df2.columns.values

array(['CID', 'TYPE', 'FIELD', 'zHD', 'zHDERR', 'MWEBV', 'HOST_NMATCH',
       'HOST_NMATCH2', 'HOST_OBJID', 'HOST_ZPHOT', 'HOST_ZPHOTERR',
       'HOST_ZSPEC', 'HOST_ZSPECERR', 'HOST_RA', 'HOST_DEC',
       'HOST_ANGSEP', 'HOST_DDLR', 'HOST_CONFUSION', 'HOST_LOGMASS',
       'HOST_LOGMASS_ERR', 'SNRMAX1', 'SNRMAX2', 'SNRMAX3',
       'TRANSIENT_NAME', 'RA', 'DEC', 'nSNR3_Y0', 'nSNR3_Y1', 'nSNR3_Y2',
       'nSNR3_Y3', 'nSNR3_Y4', 'nSNR3_Y5', 'yearSNR3', 'flagSNR3',
       'nSNR5_Y0', 'nSNR5_Y1', 'nSNR5_Y2', 'nSNR5_Y3', 'nSNR5_Y4',
       'nSNR5_Y5', 'yearSNR5', 'flagSNR5', 'cumSNR_Y0', 'cumSNR_Y1',
       'cumSNR_Y2', 'cumSNR_Y3', 'cumSNR_Y4', 'cumSNR_Y5', 'cumSNRsgn_Y0',
       'cumSNRsgn_Y1', 'cumSNRsgn_Y2', 'cumSNRsgn_Y3', 'cumSNRsgn_Y4',
       'cumSNRsgn_Y5', 'VAR0_Y0', 'VAR0_Y1', 'VAR0_Y2', 'VAR0_Y3',
       'VAR0_Y4', 'VAR0_Y5', 'VAR1_Y0', 'VAR1_Y1', 'VAR1_Y2', 'VAR1_Y3',
       'VAR1_Y4', 'VAR1_Y5', 'VARX_Y0', 'VARX_Y1', 'VARX_Y2', 'VARX_Y3',
       'VARX_Y4', 'VARX_Y5', 'minm

# Pass this onto ApplyCuts.ipynb